# BD연동 
- DB연동 하는 것은 에러가 없어야 함
- 해당 코드는 무조건 돌아가게끔 만들어야함
- MySQL에 문제가 생김
  + 대처방안 : try-catch 구문
  

In [1]:
numerator = 10
denominator = 0

result = numerator / denominator

#하고싶은 것은 결화확인 출력을 꼭 하고 싶음, 어떤일이 있어도
print("결과확인")

ZeroDivisionError: division by zero

In [2]:
try :
    numerator = 10
    denominator = 0
    
    result = numerator / denominator
except ZeroDivisionError:
    print("Error : 0으로 나눌 수 없음")
else:
    print("결과확인 : ",result)
finally:
    print("결과확인")

Error : 0으로 나눌 수 없음
결과확인


In [4]:
try :
    numerator = 10
    denominator = int(input("숫자를 입력하세요!!"))
    
    result = numerator / denominator
except ZeroDivisionError:
    print("Error : 0으로 나눌 수 없음")
except ValueError:
    print("Error : invalid literal for int() with")
else:
    print("결과확인 : ",result)
finally:
    print("결과확인")

숫자를 입력하세요!! aaa


Error : invalid literal for int() with
결과확인


## Step 1. Connect to Server

In [8]:
import os
from mysql import connector

PASSWORD = '1234'

- 기초문법 중급레벨 (with vs python decorator)

In [10]:
try:
    with connector.connect(
        host = 'localhost',
        user = 'root',
        password = PASSWORD
    ) as database:
        print(f"Database 객체 : {database}")
except connector.Error as e:
    print(e)
        

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x000002949FB508D0>


## Step 2. Creating new DB

In [16]:
try:
    with connector.connect(
        host = 'localhost', 
        user = 'root', 
        password = PASSWORD
    ) as database:
        print(f"Database 객체 : {database}")
        
        with database.cursor() as cursor:
            # 쿼리문 작성
            # create_db = "CREATE DATABASE book_ratings;"
            # cursor.execute(create_db)

            # 생성된 DB 확인 쿼리문 쓰기
            show_existing_db = "SHOW DATABASES;"
            cursor.execute(show_existing_db)
            for db in cursor:
                print(db)

except connector.Error as e:
    print(e)
    

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x000002949FB42D90>
('book_ratings',)
('classicmodels',)
('information_schema',)
('instacart',)
('modeldb',)
('mydata',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('testdb',)
('titanic',)
('world',)


In [ ]:
- 특정 DB에 직접 연결하겠다

In [18]:
try:
    with connector.connect(
        host = 'localhost',
        user = 'root',
        password = PASSWORD,
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x000002949FB2F910>


In [ ]:
## Step 3. Create DB Table

In [20]:
create_books_table = """
    CREATE TABLE books(
        id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        title VARCHAR(100),
        author VARCHAR(100),
        genre VARCHAR(100),
        release_year YEAR(4)
    );
"""

try:
    with connector.connect(
        host = 'localhost', 
        user = 'root', 
        password = PASSWORD, 
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")

        # cursor object 접근 후, 위 쿼리 실행
        with database.cursor() as cursor:
            cursor.execute(create_books_table)
            database.commit()

            # 테이블 확인 DESCRIBE
            describe_books = "DESCRIBE books;"
            cursor.execute(describe_books)
            book_schema = cursor.fetchall()
            for column in book_schema:
                print(column)
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x000002949FB42D50>
('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('title', 'varchar(100)', 'YES', '', None, '')
('author', 'varchar(100)', 'YES', '', None, '')
('genre', 'varchar(100)', 'YES', '', None, '')
('release_year', 'year', 'YES', '', None, '')


## Step 4. Data Insertion
- 하나의 레코드를 추라할 때와 다중 레코드 추가할 때 사용하는 메서드 다름


In [ ]:
# INSERT INTO 테이블명 (컬럼명1, 컬럼명2, 컬럼명3,...)

In [22]:
insert_single_record = "INSERT INTO books (id, title, author, genre, release_year)\
    VALUES (%s, %s, %s, %s, %s)"
single_record = (
    "1", "Choose Yourself! Be Happy, Make Millions, Live the Dream", "James Altucher", "self-help", "2013"
    )

In [23]:
try:
    with connector.connect(
        host = 'localhost',
        user = 'root',
        password = PASSWORD,
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")

        #커서 생성하고, 쿼리 실행
        with database.cursor() as cursor:
            cursor.execute(insert_single_record, single_record)
            database.commit()
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x000002949F55A7D0>


In [ ]:
- 다중 레코드를 추가하자

In [29]:
insert_multiple_records = "INSERT INTO books (id, title, author, genre, release_year)\
    VALUES (%s, %s, %s, %s, %s)"
multiple_records = [
    (
        "2", 
        "Skip the Line: The 10,000 Experiments Rule and Other Surprising Advice for Reaching Your Goals",
        "James Altucher",
        "self-help",
        "2021"        
    ),
    (
        "3",
        "The Power of No: Because One Little Word Can Bring Health, Abundance, and Happiness",
        "James Altucher",
        "self-help",
        "2014"
    ),
    (
        "4",
        "The 48 Laws of Power",
        "Robert Greene",
        "self-help",
        "1998"
    ),
    (
        "5",
        "Mastery",
        "Robert Greene",
        "self-help",
        "2012"
    ),
    (
        "6",
        "The Art of Seduction",
        "Robert Greene",
        "self-help",
        "2001"
    ),
]


In [31]:
try:
    with connector.connect(
        host = 'localhost',
        user = 'root',
        password = PASSWORD,
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")

        #커서 생성하고, 쿼리 실행
        with database.cursor() as cursor:
           # cursor.execute(insert_single_record, single_record)
            cursor.executemany(insert_multiple_records, multiple_records)
            database.commit()
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x00000294A04427D0>


## Step 5. Selecting Data

In [44]:
try:
    with connector.connect(
        host = 'localhost',
        user = 'root',
        password = PASSWORD,
        database = "book_ratings", # 이것이 기존 코드와의 차이점
       
    ) as database:
        print(f"Database 객체 : {database}")

        #커서 생성하고, 쿼리 실행
        with database.cursor() as cursor:

            # 쿼리문 작성
            SELECT_cols = "SELECT author, release_year From Books;"
            cursor.execute(SELECT_cols)
            # 데이터 확인 
            df = cursor.fetchall()
            for result in df:
                print(result)
            
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x00000294C8F399D0>
('James Altucher', 2013)
('James Altucher', 2021)
('James Altucher', 2014)
('Robert Greene', 1998)
('Robert Greene', 2012)
('Robert Greene', 2001)


## SELECT 사용자 정의 함수 만들기
- 어떤 테이블을 조회해도 깔끔하게 데이터 프레임으로 만드는 함수를 만드시고, 
- 테스트 해주세요


In [48]:
import pandas as pd

def fetch_data_to_dataframe(query, database=None):
    try: 
        with connector.connect(
            host="localhost",
            user="root",
            password=PASSWORD,
            database=database
        ) as existing_database:
            
            with existing_database.cursor() as cursor:
                cursor.execute(query)
                
                column_names = [i[0] for i in cursor.description] # List Comprehension
                
                returned_data = cursor.fetchall()
                
                df = pd.DataFrame(returned_data, columns=column_names)
                return df
    
    except connector.Error as e: 
        print(e)
        return None

In [ ]:
query_all_cols = "SELECT * FROM dataset2"
DATABASE = "mydata"
df_all_cols = fetch_data_to_dataframe(query_

# Step 6.Update and Delete

In [49]:
update_query = """
    UPDATE 
        books
    SET 
        author = 'Big J'
    WHERE 
        author = 'James Altucher'
"""

try:
    with connector.connect(
        host = 'localhost', 
        user = 'root', 
        password = PASSWORD, 
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")

        # 커서 생성 후, 쿼리 실행
        with database.cursor() as cursor:
            cursor.execute(update_query)
            database.commit()

            check_query = "SELECT DISTINCT author FROM books"
            cursor.execute(check_query)

            for result in cursor.fetchall():
                print(result)
            
except connector.Error as e:
    print(e)
    

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x00000294C8F4A890>
('Big J',)
('Robert Greene',)


# Step 7. Delete Record

In [50]:
try:
    with connector.connect(
        host = 'localhost', 
        user = 'root', 
        password = PASSWORD, 
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")

        # 커서 생성 후, 쿼리 실행
        with database.cursor() as cursor:

            delete_query = "DELETE FROM books WHERE release_year <= 2010;"
            cursor.execute(delete_query)
            database.commit()

            check_query = "SELECT DISTINCT release_year FROM books;"
            cursor.execute(check_query)
            for result in cursor.fetchall():
                print(result)
            
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x00000294C8F2F890>
(2013,)
(2021,)
(2014,)
(2012,)
